# **非構造化RAG**
非構造化または（準構造化）RAGは、テキスト、表、画像を組み合わせた文書を処理するために設計された手法です。テキストの分割による表の分断や、意味検索のための表の埋め込みの難しさといった課題に対応します。

ここでは、テキスト、表、画像を解析して分離するために、`unstructured.io`を使用しています。

**ツール参照:** [Unstructured](https://unstructured.io/)

---
## 環境構築 (macOS)

以下のツールを環境にインストールします。

```
brew install libmagic poppler libreoffice pandoc tesseract
```

`unstructured` のインストール:
*   [unstructured.io インストールガイド](https://docs.unstructured.io/open-source/installation/full-installation)

```
pip install "unstructured[all-docs]"
```

**対応ドキュメント形式:**

"csv", "doc", "docx", "epub", "image", "md", "msg", "odt", "org", "pdf", "ppt", "pptx", "rtf", "rst", "tsv", "xlsx"

---

## 特定のデータコネクタのインストール (例: S3):

```
pip install "unstructured-ingest[s3]" # 今回は不使用
```

**データコネクタとは:**

様々なデータソースからデータを取得し、アプリケーションやシステムで利用できるようにするためのツールまたはコンポーネントです。データの形式やアクセス方法の違いを吸収します。

**対応データコネクタ:**
"airtable", "azure", "azure-ai-search", "biomed", "box", "confluence", "couchbase", "delta-table", "discord", "dropbox", "elasticsearch", "gcs", "github", "gitlab", "google-drive", "jira", "mongodb", "notion", "opensearch", "onedrive", "outlook", "reddit", "s3", "sharepoint", "salesforce", "slack", "wikipedia"



## **🔸 Multimodal Embedding を使いたい場合（画像+テキスト）**
🔹 **現時点で Gemini AI Studio API ではマルチモーダルの埋め込みは未対応！**  
🔹 **GCP の Vertex AI API を使う必要があるのだ！**  

| **方法** | **Gemini AI Studio API** | **Vertex AI API (GCP)** |
|------------|-------------------|----------------|
| **テキスト埋め込み** | ✅ 可能（`models/embedding-001`） | ✅ 可能（`textembedding-gecko`） |
| **画像の埋め込み** | ❌ 未対応 | ❌ 未対応 |
| **画像 + テキスト埋め込み** | ❌ 直接は不可（画像→テキスト→埋め込み） | ❌ 直接は不可 |
| **商用利用** | ❌ 制限あり | ✅ 可能 |
| **おすすめ用途** | ✅ 個人開発 & プロトタイプ | ✅ 企業向け & 本番環境 |

🚀 **結論**  
🔹 **「Gemini AI Studio API」では、画像の埋め込みは直接できないが、画像 → テキスト → 埋め込み の方法なら可能！**  
🔹 **マルチモーダル埋め込みが必要なら、OpenAI CLIP など他の選択肢も検討するのだ！**


In [76]:
!pip install -qU vertexai langchain_google_vertexai "unstructured[all-docs]" pdfplumber

In [ ]:
# !gcloud auth application-default set-quota-project ${project_id}
# !gcloud config get-value project


# PDFから画像、テーブル、チャンク化テキストを読み込む

In [79]:
# load and extract images, tables, and chunk text
from unstructured.partition.pdf import partition_pdf
import os

dataset_dir = "../data/pdf/"
pdf_file_name = "Attention Is All You Need 1706.03762v7.pdf"
extract_image_block_output_dir = os.path.join(dataset_dir, "images")
os.makedirs(extract_image_block_output_dir, exist_ok=True)

# pdf_elements = partition_pdf(
#         filename=os.path.join(dataset_dir, pdf_file_name),
#         extract_images_in_pdf=True,
#         infer_table_structure=True,
#         chunking_strategy="by_title",
#         max_characters=4000,
#         new_after_n_chars=3800,
#         combine_text_under_n_chars=2000,
#         image_output_dir_path=os.path.join(dataset_dir, "images"),
#     )

pdf_elements = partition_pdf(
    filename=os.path.join(dataset_dir, pdf_file_name),
    extract_images_in_pdf=True,
    infer_table_structure=True,
    # max_characters=4000,
    # combine_text_under_n_chars=200,
    extract_image_block_output_dir=extract_image_block_output_dir, # defaultでは実行ファイルがあるフォルダ下にfiguresが作成される
)


## [unstructured.partition.pdf のパラメータ](https://docs.unstructured.io/open-source/core-functionality/partitioning)

### 主なパラメータ

| パラメータ | 説明 |
|---|---|
| `filename` | PDF ファイルのパスを指定します。 |
| `file` | PDF ファイルオブジェクトを指定します。`filename` または `file` のいずれかを指定する必要があります。 |
| `strategy` | PDF ファイルの分割戦略を指定します。 |
|  | `"auto"`: 自動的に最適な戦略を選択します (デフォルト)。 |
|  | `"fast"`: 高速な分割を行います。 |
|  | `"hi_res"`: 高解像度な分割を行います。 |
|  | `"ocr_only"`: OCR (光学文字認識) のみを行います。 |
| `infer_table_structure` | テーブル構造を推論するかどうかを指定します (`True`/`False`)。 |
| `ocr_languages` | OCR で使用する言語を指定します。 |
| `encoding` | ファイルのエンコーディングを指定します。 |
| `include_page_breaks` | ページ区切りを含めるかどうかを指定します (`True`/`False`)。 |
| `max_partition` | 最大分割数を指定します。 |
| `extract_image_block_output_dir` | 画像の出力フォルダを指定します。 |

### 詳細なパラメータ

| パラメータ | 説明 |
|---|---|
| `chunking_strategy` | 分割された要素をどのようにチャンクにまとめるかを指定します。 |
|  | `"by_title"`: タイトルごとにチャンクをまとめます。 |
|  | `"by_paragraph"`: 段落ごとにチャンクをまとめます。 |
|  | `"contiguous"`: 連続する要素をまとめてチャンクにします。 |
| `new_after_n_words` | チャンクの最大単語数を指定します。 |
| `new_after_n_pages` | チャンクの最大ページ数を指定します。 |
| `metadata_filename` | メタデータファイルを指定します。 |
| `pdf_text_extraction` | PDF テキスト抽出方法を指定します。 |
|  | `"textract"`: textract ライブラリを使用します。 |
|  | `"tika"`: Tika サーバーを使用します。 |
|  | `"paddlepaddle"`: PaddlePaddle OCR を使用します。 |

### パラメータの選択例

| 例 | 設定 |
|---|---|
| 高速な分割 | `strategy="fast"` |
| 高解像度な分割 | `strategy="hi_res"` |
| テーブル構造の推論 | `infer_table_structure=True` |
| 英語と日本語の OCR | `ocr_languages=["en", "ja"]` |
| ページ区切りを含める | `include_page_breaks=True` |

### 注意点

* パラメータの組み合わせによっては、期待通りの結果が得られない場合があります。
* PDF ファイルの内容や構造によっては、分割がうまくいかない場合があります。
* OCR を使用する場合、言語によっては精度が低い場合があります。

In [80]:
# # check unique categories

from collections import Counter
category_counts = Counter(str(type(element)) for element in pdf_elements)
unique_categories = set(category_counts)
category_counts

Counter({"<class 'unstructured.documents.elements.NarrativeText'>": 83,
         "<class 'unstructured.documents.elements.ListItem'>": 43,
         "<class 'unstructured.documents.elements.Title'>": 38,
         "<class 'unstructured.documents.elements.Text'>": 21,
         "<class 'unstructured.documents.elements.Footer'>": 8,
         "<class 'unstructured.documents.elements.Image'>": 7,
         "<class 'unstructured.documents.elements.FigureCaption'>": 5,
         "<class 'unstructured.documents.elements.Formula'>": 5,
         "<class 'unstructured.documents.elements.Table'>": 4,
         "<class 'unstructured.documents.elements.Header'>": 3,
         "<class 'unstructured.documents.elements.EmailAddress'>": 1})

In [81]:
# extract unique types
unique_types = {el.to_dict()['type'] for el in pdf_elements}
unique_types

{'EmailAddress',
 'FigureCaption',
 'Footer',
 'Formula',
 'Header',
 'Image',
 'ListItem',
 'NarrativeText',
 'Table',
 'Title',
 'UncategorizedText'}

In [91]:
# multi-modal-RAGにおいてgeminiのapiを使うにはGCPで(面倒な)設定をする必要がある様子。
# こちらを参考にした。https://github.com/shohei0990/Multimodal_RAG_00/tree/main?tab=readme-ov-file

import os
from dotenv import load_dotenv
load_dotenv()

import vertexai
import google.generativeai as genai
from vertexai.language_models import TextEmbeddingModel

# 環境変数 GOOGLE_APPLICATION_CREDENTIALS を設定
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getenv("gac1") # gac2でもいけた
vertexai.init(project=os.getenv("gcp_project_id"), location="us-central1")

# Embedding モデルをロード
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")

# テキストを埋め込む
texts = ["AI is evolving.", "Machine learning is powerful."]
embeddings = model.get_embeddings(texts)

# 結果を表示
for i, emb in enumerate(embeddings):
    print(f"Text {i+1}: {texts[i]}")
    print(f"Embedding Vector (first 5 dims): {emb.values[:5]}")


In [96]:
from langchain_google_vertexai import VertexAIEmbeddings

model_name = "multimodalembedding@latest"
embeddings = VertexAIEmbeddings(model_name=model_name, project="my-project-20250209-450405", location="us-central1")

print("Vertex AI Embeddings initialized successfully!")



PermissionDenied: 403 Project gen-lang-client-0318295445 has been deleted. [reason: "USER_PROJECT_DENIED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gen-lang-client-0318295445"
}
metadata {
  key: "consumer"
  value: "projects/gen-lang-client-0318295445"
}
, locale: "en-US"
message: "Project gen-lang-client-0318295445 has been deleted."
]

In [ ]:
# load embedding model
from langchain_google_vertexai import VertexAIEmbeddings

model_name = "multimodalembedding@latest"
embeddings = VertexAIEmbeddings(model_name=model_name, project=os.getenv("gcp_project_id"))

# convert pdf_elements to langchain documents
from langchain.schema import Document
documents = [Document(page_content=el.text, metadata={'source': fileitem}) for el in pdf_elements]

# create vectorstore
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings)

# retriever
retriever = vectorstore.as_retriever()

llm = gemini # response = gemini.generate_content(prompt), response.text

In [ ]:
# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the provided context, which can include text and tables.
Use the provided context to answer the question.
Question: {input}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# response
response = rag_chain.invoke("Compare all the Training Results on MATH Test Set")
response